https://github.com/lsjsj92/recommender_system_with_Python/blob/master/004.%20recommender%20system%20basic%20with%20Python%20-%203%20Matrix%20Factorization.ipynb

In [1]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 문제 정보만 가져온다
problem_data = pd.read_csv("final_data/CSV/recommend_problem_infos.csv")

# problem_data = problem_data.iloc[:,:11]
# problem_data['problem_id'] = range(len(problem_data))

problem_data.info()
problem_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9267 entries, 0 to 9266
Data columns (total 13 columns):
seq               9267 non-null int64
number            9267 non-null int64
name              9267 non-null object
algorithm         9267 non-null object
algorithm_id      9267 non-null object
information       9267 non-null object
correct_user      9267 non-null int64
submission_cnt    9267 non-null int64
correct_rate      9267 non-null float64
level             9267 non-null int64
avg_try           9267 non-null float64
time_limit        9267 non-null object
memory_limit      9267 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 941.3+ KB


,seq,number,name,algorithm,algorithm_id,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit
0,0,1000,A+B,"수학,구현,사칙연산","124,102,121","다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB
1,1,1001,A-B,"수학,구현,사칙연산","124,102,121","디버그,분류",97109,159168,72.049,1,1.39,2 초,128 MB
2,2,10998,A×B,"수학,구현,사칙연산","124,102,121","디버그,분류",69367,97906,78.356,1,1.28,1 초,256 MB
3,3,10869,사칙연산,"수학,구현,사칙연산","124,102,121","디버그,분류",68791,150720,52.485,1,1.90,1 초,256 MB
4,4,1008,A/B,"수학,구현,사칙연산","124,102,121","스페셜 저지,분류",67564,248349,33.473,2,2.99,2 초,128 MB


In [3]:
ranker_problems = pd.read_csv("final_data/CSV/ranker_problems.csv")

ranker_problems.info()
ranker_problems.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6148089 entries, 0 to 6148088
Data columns (total 7 columns):
id            int64
user_id       int64
user          object
number        int64
TF            object
problem_id    int64
score         int64
dtypes: int64(5), object(2)
memory usage: 328.3+ MB


,id,user_id,user,number,TF,problem_id,score
0,0,0,koosaga,1000,o,0,1
1,1,0,koosaga,1001,o,1,1
2,2,0,koosaga,1002,o,45,1
3,3,0,koosaga,1003,o,2242,1
4,4,0,koosaga,1004,o,8398,1


In [4]:
user2idx = {}
for user_id, user_name in zip(ranker_problems['user_id'], ranker_problems['user']) :
    user2idx[user_name] = user_id
len(user2idx)

34900

In [5]:
problem2idx = {}
for index, number in zip(problem_data['number'], problem_data['seq']) :
    problem2idx[index] = number
len(problem2idx)

9267

In [6]:
len(ranker_problems['number'].unique()), len(problem_data['number'].unique())

(9267, 9267)

In [7]:
score_data = ranker_problems[['user_id', 'number', 'score']]

score_data.info()
score_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6148089 entries, 0 to 6148088
Data columns (total 3 columns):
user_id    int64
number     int64
score      int64
dtypes: int64(3)
memory usage: 140.7 MB


,user_id,number,score
0,0,1000,1
1,0,1001,1
2,0,1002,1
3,0,1003,1
4,0,1004,1


In [8]:
user_problem_data = pd.merge(score_data, problem_data, on = 'number')
user_problem_data.head()

,user_id,number,score,seq,name,algorithm,algorithm_id,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit
0,0,1000,1,0,A+B,"수학,구현,사칙연산","124,102,121","다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB
1,1,1000,1,0,A+B,"수학,구현,사칙연산","124,102,121","다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB
2,2,1000,1,0,A+B,"수학,구현,사칙연산","124,102,121","다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB
3,3,1000,1,0,A+B,"수학,구현,사칙연산","124,102,121","다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB
4,4,1000,1,0,A+B,"수학,구현,사칙연산","124,102,121","다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB


In [9]:
user_problem_data.shape

(6148089, 15)

In [10]:
user_problem_score = user_problem_data.pivot_table('score', index = 'user_id', columns = 'number').fillna(0)
user_problem_score.shape

(34900, 9267)

In [11]:
user_problem_score.head()

number,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,21215,21221,21222,21226,21227,21232,21233,21236,21262,21269
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
problem_user_score = user_problem_score.values.T
problem_user_score.shape

(9267, 34900)

In [13]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(problem_user_score)
matrix.shape

(9267, 12)

In [14]:
corr = np.corrcoef(matrix)
corr.shape

(9267, 9267)

In [15]:
problem_number = user_problem_score.columns
problem_number_list = list(problem_number)
nNm = problem_number_list.index(15651)

In [16]:
corr_nNm  = corr[nNm]
list(problem_number[(corr_nNm >= 0.9)])

[2580, 9663, 12865, 15649, 15650, 15651, 15652, 17298]

In [17]:
score_data.head()

,user_id,number,score
0,0,1000,1
1,0,1001,1
2,0,1002,1
3,0,1003,1
4,0,1004,1


In [18]:
df_user_problem_scores = score_data.pivot(index = "user_id", columns = "number", values = 'score').fillna(0)
df_user_problem_scores.head()

number,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,21215,21221,21222,21226,21227,21232,21233,21236,21262,21269
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
matrix = df_user_problem_scores.as_matrix()

user_scores_mean = np.mean(matrix, axis = 1)

matrix_user_mean = matrix - user_scores_mean.reshape(-1, 1)

In [20]:
matrix

array([[1., 1., 1., ..., 0., 0., 1.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [21]:
pd.DataFrame(matrix_user_mean, columns = df_user_problem_scores.columns).head()

number,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,21215,21221,21222,21226,21227,21232,21233,21236,21262,21269
0,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,...,-0.695586,-0.695586,-0.695586,0.304414,-0.695586,-0.695586,-0.695586,-0.695586,-0.695586,0.304414
1,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,...,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884
2,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,...,0.475774,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226
3,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,...,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622
4,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,...,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219


In [22]:
U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [23]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(34900, 12)
(12,)
(12, 9267)


In [24]:
sigma = np.diag(sigma)

In [25]:
sigma.shape

(12, 12)

In [26]:
sigma[0], sigma[1]

(array([178.85121295,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ]),
 array([  0.        , 198.50825048,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ]))

In [27]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_scores_mean.reshape(-1, 1)

In [28]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_problem_scores.columns)
df_svd_preds.head()

number,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,21215,21221,21222,21226,21227,21232,21233,21236,21262,21269
0,1.092643,1.161317,0.940160,0.995191,1.091377,1.296092,0.955534,0.959898,1.225682,1.054703,...,0.593172,0.594564,0.592707,0.592895,0.591426,0.591660,0.592336,0.593313,0.594595,0.593421
1,1.199842,1.209776,0.630115,0.485913,0.951153,1.372896,0.978107,1.021602,1.216834,0.589947,...,0.443747,0.446063,0.442593,0.442180,0.439982,0.440925,0.441569,0.443817,0.445833,0.443297
2,1.164612,1.132237,0.682601,0.727076,0.859683,1.087092,0.776338,0.829634,1.103832,0.679217,...,0.357684,0.360417,0.356749,0.356056,0.354249,0.355035,0.355652,0.357928,0.359590,0.357017
3,1.027797,1.066543,0.969558,0.663756,1.052454,1.456662,0.840841,0.875087,1.093395,0.980097,...,0.342221,0.343483,0.341485,0.341329,0.339838,0.340599,0.340779,0.342156,0.343736,0.342085
4,1.153425,1.177639,0.611948,0.251381,0.955108,1.506327,0.943701,1.001102,1.193697,0.565558,...,0.279569,0.282073,0.278250,0.277779,0.275266,0.276456,0.277055,0.279586,0.282008,0.279084


In [29]:
df_svd_preds.shape

(34900, 9267)

In [30]:
def recommend_problems(df_svd_preds, user_id, ori_problem_df, ori_scores_df, num_recommendations = 5):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[user_id].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_scores_df[ori_scores_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_problem_df, on = 'number').sort_values(['score'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_problem_df[~ori_problem_df['number'].isin(user_history['number'])]
    
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'number')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [31]:
already_rated, predictions = recommend_problems(df_svd_preds, 5258, problem_data, score_data, 10)

In [32]:
already_rated.head(10)

,user_id,number,score,seq,name,algorithm,algorithm_id,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit
255,5258,15998,2,677,카카오머니,수학,124,"스페셜 저지,출처,분류",184,3131,10.127,12,9.88,5 초 (추가 시간 없음),256 MB
254,5258,9935,2,6085,문자열 폭발,"자료 구조,문자열,스택","175,158,71","출처,다국어,분류",3672,26433,21.764,12,4.59,2 초 (추가 시간 없음),128 MB
253,5258,5577,2,4317,RBY팡!,"구현,시뮬레이션","102,141","출처,다국어,분류",102,480,29.565,14,3.38,1 초,128 MB
252,5258,2098,2,2284,외판원 순회,"다이나믹 프로그래밍,비트마스킹,비트필드를 이용한 다이나믹 프로그래밍,외판원 순회 문제","25,14,87,138",분류,3585,22494,28.595,15,3.50,1 초,128 MB
251,5258,1167,2,4947,트리의 지름,"그래프 이론,그래프 탐색,트리,깊이 우선 탐색","7,11,120,127",분류,4189,15823,35.636,13,2.81,2 초,256 MB
250,5258,1059,2,282,좋은 구간,"수학,브루트포스 알고리즘","124,125",분류,1080,5229,25.924,6,3.86,2 초,128 MB
167,5258,10950,1,10,A+B - 3,"수학,구현,사칙연산","124,102,121",분류,47583,94066,58.953,3,1.70,1 초,256 MB
172,5258,10973,1,132,이전 순열,수학,124,분류,3744,7140,65.489,8,1.53,1 초,256 MB
171,5258,10972,1,101,다음 순열,"수학,조합론","124,6",분류,4909,16119,43.331,8,2.31,1 초,256 MB
170,5258,10971,1,8695,외판원 순회 2,"백트래킹,외판원 순회 문제","5,138",분류,5457,25439,34.747,9,2.88,2 초,256 MB


In [33]:
predictions

,seq,number,name,algorithm,algorithm_id,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,Predictions
7467,7688,9663,N-Queen,"브루트포스 알고리즘,백트래킹","125,5",분류,12973,36655,53.286,11,1.88,10 초,128 MB,0.890499
4758,4918,4963,섬의 개수,"그래프 이론,그래프 탐색,너비 우선 탐색,깊이 우선 탐색","7,11,126,127","출처,다국어,분류",10690,29537,49.204,9,2.03,1 초,128 MB,0.871751
4760,4925,7562,나이트의 이동,"그래프 이론,그래프 탐색,너비 우선 탐색","7,11,126","출처,다국어,분류",8335,23165,46.923,9,2.13,1 초,256 MB,0.808738
4759,4923,2583,영역 구하기,"그래프 이론,그래프 탐색,너비 우선 탐색,깊이 우선 탐색","7,11,126,127","출처,분류",9034,20439,56.615,10,1.77,1 초,128 MB,0.765276
5,22,15552,빠른 A+B,"수학,구현,사칙연산","124,102,121",분류,37056,97596,46.940,4,2.13,1 초 (하단 참고),512 MB,0.738347
5868,6063,1158,요세푸스 문제,"자료 구조,큐","175,72",분류,14146,39824,49.209,6,2.03,2 초,256 MB,0.729984
1,12,1110,더하기 사이클,"수학,구현,문자열","124,102,158",분류,44365,111142,48.021,5,2.08,2 초,128 MB,0.716441
0,7,8393,합,"수학,구현","124,102","출처,다국어,분류",58154,97631,68.396,1,1.46,1 초,128 MB,0.714595
3595,3698,7568,덩치,"구현,브루트포스 알고리즘","102,125","출처,분류",15518,31086,58.694,6,1.70,1 초,128 MB,0.688476
3585,3674,10817,세 수,구현,102,분류,41355,104998,46.811,3,2.14,1 초,256 MB,0.687393


In [34]:
user2idx['kkwinwin95'], user2idx['esun1903'], user2idx['pyob32']

(5258, 10170, 5238)

In [35]:
already_rated_k, predictions_k = recommend_problems(df_svd_preds, 5258, problem_data, score_data, 10)
predictions_k

,seq,number,name,algorithm,algorithm_id,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,Predictions
7467,7688,9663,N-Queen,"브루트포스 알고리즘,백트래킹","125,5",분류,12973,36655,53.286,11,1.88,10 초,128 MB,0.890499
4758,4918,4963,섬의 개수,"그래프 이론,그래프 탐색,너비 우선 탐색,깊이 우선 탐색","7,11,126,127","출처,다국어,분류",10690,29537,49.204,9,2.03,1 초,128 MB,0.871751
4760,4925,7562,나이트의 이동,"그래프 이론,그래프 탐색,너비 우선 탐색","7,11,126","출처,다국어,분류",8335,23165,46.923,9,2.13,1 초,256 MB,0.808738
4759,4923,2583,영역 구하기,"그래프 이론,그래프 탐색,너비 우선 탐색,깊이 우선 탐색","7,11,126,127","출처,분류",9034,20439,56.615,10,1.77,1 초,128 MB,0.765276
5,22,15552,빠른 A+B,"수학,구현,사칙연산","124,102,121",분류,37056,97596,46.940,4,2.13,1 초 (하단 참고),512 MB,0.738347
5868,6063,1158,요세푸스 문제,"자료 구조,큐","175,72",분류,14146,39824,49.209,6,2.03,2 초,256 MB,0.729984
1,12,1110,더하기 사이클,"수학,구현,문자열","124,102,158",분류,44365,111142,48.021,5,2.08,2 초,128 MB,0.716441
0,7,8393,합,"수학,구현","124,102","출처,다국어,분류",58154,97631,68.396,1,1.46,1 초,128 MB,0.714595
3595,3698,7568,덩치,"구현,브루트포스 알고리즘","102,125","출처,분류",15518,31086,58.694,6,1.70,1 초,128 MB,0.688476
3585,3674,10817,세 수,구현,102,분류,41355,104998,46.811,3,2.14,1 초,256 MB,0.687393


In [36]:
# 은선
already_rated_c, predictions_c = recommend_problems(df_svd_preds, 10170, problem_data, score_data, 10)
predictions_c

,seq,number,name,algorithm,algorithm_id,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,Predictions
7534,7683,1065,한수,브루트포스 알고리즘,125,분류,29974,68029,51.984,7,1.92,2 초,128 MB,0.777182
3599,3683,2750,수 정렬하기,"구현,정렬","102,97",분류,30976,78735,58.145,5,1.72,1 초,128 MB,0.703698
7857,8018,2751,수 정렬하기 2,정렬,97,분류,20294,114927,30.102,6,3.32,2 초,256 MB,0.692572
1,27,4673,셀프 넘버,"수학,구현","124,102","출처,다국어,분류",29207,72518,50.260,5,1.99,1 초,256 MB,0.690241
3604,3695,2941,크로아티아 알파벳,"구현,문자열","102,158","출처,다국어,분류",20778,54801,45.344,6,2.21,1 초,128 MB,0.689267
3603,3694,1316,그룹 단어 체커,"구현,문자열","102,158",분류,21685,49239,52.359,6,1.91,2 초,128 MB,0.661100
6817,6964,11399,ATM,"그리디 알고리즘,정렬","33,97",분류,21087,39255,65.971,8,1.52,1 초,256 MB,0.652726
4795,4926,2206,벽 부수고 이동하기,"그래프 이론,그래프 탐색,너비 우선 탐색","7,11,126",분류,7909,52376,22.644,12,4.42,2 초,192 MB,0.630513
6818,6965,11047,동전 0,그리디 알고리즘,33,"디버그,분류",20005,47672,52.837,9,1.89,1 초,256 MB,0.628558
6819,6966,1931,회의실 배정,"그리디 알고리즘,정렬","33,97",분류,15557,73088,28.661,9,3.49,2 초,128 MB,0.614110


In [37]:
# 기동
already_rated_p, predictions_p = recommend_problems(df_svd_preds, 5238, problem_data, score_data, 10)
predictions_p

,seq,number,name,algorithm,algorithm_id,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,Predictions
6741,6965,11047,동전 0,그리디 알고리즘,33,"디버그,분류",20005,47672,52.837,9,1.89,1 초,256 MB,0.868951
3613,3725,14891,톱니바퀴,"구현,시뮬레이션","102,141","디버그,분류",6863,18308,52.469,11,1.91,2 초,512 MB,0.836279
3607,3718,14499,주사위 굴리기,"구현,시뮬레이션","102,141","디버그,분류",7941,26536,41.896,11,2.39,2 초,512 MB,0.830731
4758,4925,7562,나이트의 이동,"그래프 이론,그래프 탐색,너비 우선 탐색","7,11,126","출처,다국어,분류",8335,23165,46.923,9,2.13,1 초,256 MB,0.823444
3611,3723,13460,구슬 탈출 2,"구현,그래프 이론,그래프 탐색,너비 우선 탐색","102,7,11,126","디버그,분류",7063,46185,25.719,14,3.89,2 초,512 MB,0.822167
3619,3734,16234,인구 이동,"구현,그래프 이론,그래프 탐색,너비 우선 탐색,시뮬레이션","102,7,11,126,141","디버그,분류",5705,25545,36.225,11,2.76,2 초,512 MB,0.821823
3614,3726,12100,2048 (Easy),"구현,브루트포스 알고리즘,시뮬레이션","102,125,141","디버그,분류",6593,43459,24.497,14,4.08,1 초,512 MB,0.801340
5855,6063,1158,요세푸스 문제,"자료 구조,큐","175,72",분류,14146,39824,49.209,6,2.03,2 초,256 MB,0.789961
3599,3703,1966,프린터 큐,"구현,자료 구조,시뮬레이션,큐","102,175,141,72","출처,다국어,분류",11559,27320,55.296,8,1.81,2 초,128 MB,0.778065
7959,8216,2805,나무 자르기,이분 탐색,12,"출처,다국어,분류",11360,63044,25.940,8,3.85,1 초,256 MB,0.767100
